In [2]:
import bambi as bmb
import arviz as az
import pymc as pm
import pandas as pd

In [26]:
arr_zinb = pm.ZeroInflatedBinomial.dist(p=.30, n=100, psi=.7, size=100).eval()

df_zinb = pd.DataFrame(arr_zinb, columns=['hits']).assign(at_bats=100)

df_zinb.head()

,hits,at_bats
0,33,100
1,30,100
2,0,100
3,33,100
4,0,100


In [67]:
likelihood = bmb.Likelihood("ZeroInflatedBinomial", params=["p", "psi"], parent="p")
links = {"p": "logit", "psi": "logit"}
zinb_family = bmb.Family("zinb", likelihood, links)
zinb_family

Family: zinb
Likelihood: Likelihood(  
  name: ZeroInflatedBinomial,
  params: ['p', 'psi'],
  parent: p
)
Link: {'p': Link(  
  name: logit,
  link: <function logit at 0x29b4534c0>,
  linkinv: <function expit at 0x29b453420>
), 'psi': Link(  
  name: logit,
  link: <function logit at 0x29b4534c0>,
  linkinv: <function expit at 0x29b453420>
)}

In [69]:
priors = {"psi": bmb.Prior("Normal", mu=0, sigma=2)}

model = bmb.Model("p(hits, at_bats) ~ 1", df_zinb, family=zinb_family, priors=priors)

model

       Formula: p(hits, at_bats) ~ 1
        Family: zinb
          Link: p = logit
  Observations: 100
        Priors: 
    target = p
        Common-level effects
            Intercept ~ Normal(mu: 0.0, sigma: 2.5)
        
        Auxiliary parameters
            p(hits, at_bats)_psi ~ Normal(mu: 0.0, sigma: 2.0)

In [72]:
idata = model.fit()

idata

TypeError: ZeroInflatedBinomial.__new__() missing 1 required positional argument: 'n'